In [1]:
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler
from keras.models import load_model
import os 


Using TensorFlow backend.


In [2]:
def rolling_window(a, window, step_size, padding=True, copy=True):
    if copy:
        result = a.copy()
    else:
        result = a
    if padding:
        result = np.hstack((result, np.zeros(window)))
    shape = result.shape[:-1] + (result.shape[-1] - window + 1 - step_size, window)
    strides = result.strides + (result.strides[-1] * step_size,)
    return np.lib.stride_tricks.as_strided(result, shape=shape, strides=strides)

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from obspy.core import read, UTCDateTime
from obspy.io.sac.util import get_sac_reftime
from obspy.signal.filter import lowpass, bandpass, highpass
import datetime


def picker_p_predict(file, scaler, normalizer, file_cnn, size):
    cnn = load_model(file_cnn)
    p_is_defined = False
    tr = read(file)[0]
    if tr.stats.npts >= size :
        tr.filter('bandpass', freqmin=1.0, freqmax=10.0, corners=4, zerophase=True)
        
        
        if 'a' in tr.stats.sac and tr.stats.sac.a is not None: 
            picker_p = int(tr.stats.sac.a * tr.stats.sampling_rate)
            p_is_defined = True
        
        data = tr.data

        #Escalar datos antes de hacer el window
        if scaler == 'min_max_scaler':
            scaler = MinMaxScaler()
        elif scaler == 'standard_scaler':
            scaler = StandardScaler()
        elif scaler == 'min_max_scaler_1':
            scaler = MinMaxScaler(feature_range=(-1, 1))
        scaler = scaler.fit(tr.data.reshape(-1, 1))
        x = scaler.transform(tr.data.reshape(1, -1))[0]
        if normalizer:
            normalizer = Normalizer().fit(tr.data.reshape(-1, 1))
            x = normalizer.transform(x.reshape(1, -1))[0]
        else:
            tr.normalize()
        
        input_dataset = rolling_window(tr.data, size, 1)

        input_dataset = np.reshape(input_dataset, (len(input_dataset), size, 1))
        output = [x[1] for x in cnn.predict(input_dataset)]
        output_running_mean = running_mean(output, size)
        guess = np.argmax(output_running_mean)


        plt.figure(figsize=(50,10))
        plt.plot(tr.times(), data)

        if p_is_defined:
            plt.axvline(x=(picker_p*tr.stats.delta) , color='red')
            print("picker_p:", picker_p)

        plt.axvline(x=(guess*tr.stats.delta) , color='green', drawstyle ="steps-pre") 
        print("guess:", guess)
        plt.xlabel('Time')
        plt.ylabel('Amplitude')

    #     plt.show()
        datetime_object = datetime.datetime.now()
        print(datetime_object)
        cnn_name = file_cnn.split("/")[6].split(".")[0] + "/"
        cnn_name = '/home/irene/Documents/Resultados/'+cnn_name
        print(cnn_name)
        if not os.path.exists(cnn_name):
            os.mkdir(cnn_name)
        file_png = cnn_name+tr.stats.station+'_'+datetime_object.strftime('%m-%d-%Y::%H-%M')+'.png'
        print(file_png)
        plt.savefig(file_png)

In [ ]:
base_path = '", "/home/irene/Documents/Datos-Tesis/Estaciones/'

files_name = ["/home/irene/Documents/Datos-Tesis/Estaciones/2010.181.195432.SAC",
              "/home/irene/Documents/Datos-Tesis/Estaciones/ACHA.HHZ.2019.088.14.45.11",
              "/home/irene/Documents/Datos-Tesis/Estaciones/BATAN.HHZ.2018.337.16.57.18.sac",
              "/home/irene/Documents/Datos-Tesis/Estaciones/BATAN.HHZ.2019.088.14.45.11",
              "/home/irene/Documents/Datos-Tesis/Estaciones/CALV.HHZ.2018.337.16.57.18.sac", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/CCOL.HHZ.2019.088.14.45.11", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/CDITO.HHZ.2019.088.14.45.11", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/CDM.HHZ.2018.337.16.57.18.sac", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/CDM.HHZ.2019.088.14.45.11", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/CNKC.HHZ.2018.337.16.57.18.sac", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/CNKC.HHZ.2019.088.14.45.11", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/COVE.HHZ.2018.337.16.57.18.sac", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/COVE.HHZ.2019.088.14.45.11", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/CPAN.HHZ.2018.337.16.57.18.sac", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/CPMI.HHZ.2019.088.14.45.11", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/CTCR.HHZ.2019.088.14.45.11",
              "/home/irene/Documents/Datos-Tesis/Estaciones/CVTQ.HHZ.2019.088.14.45.11", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/CVTR.HHZ.2019.088.14.45.11", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/DUNO.HHZ.2019.088.14.45.11", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HAYA.HHZ.2019.088.14.45.11", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-08-06T13:19:39.320000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-08-07T09:20:19.840000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-08-09T03:15:02.060000.SAC",
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-08-10T07:24:35.140000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-08-17T09:05:20.280000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-08-17T09:12:18.920000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-08-19T11:06:11.320000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-08-22T10:04:41.140000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-09-03T06:38:51.340000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-09-04T00:29:11.800000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-09-04T02:45:35.840000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-09-04T07:08:14.680000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-09-05T11:50:17.340000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-09-11T08:23:58.040000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-09-16T21:18:43.120000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-09-17T03:04:58.940000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-09-17T03:05:27.100000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-09-26T06:28:30.480000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-09-30T09:44:08.580000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-09-30T09:44:24.980000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-10-09T05:16:41.920000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-10-09T05:36:21.460000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-10-19T09:23:52.940000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2017-12-27T21:49:42.320000.SAC", 
              "/home/irene/Documents/Datos-Tesis/Estaciones/HDC3.HHZ.2018.337.16.57.18.sac"]

for file_name in files_name:
    file = os.path.join(base_path, file_name)
    print(file)
    cnn_to_run = [picker_p_predict(file, 'min_max_scaler_1', False, '/home/irene/Documents/Datos-Tesis/results/0b3c148e-977a-11e9-a0d3-f4e9d4c27a82.h5', 600), 
       picker_p_predict(file, 'min_max_scaler', True, "/home/irene/Documents/Datos-Tesis/results/01d880d6-9765-11e9-a0d3-f4e9d4c27a82.h5", 60),
       picker_p_predict(file, 'min_max_scaler_1', True, "/home/irene/Documents/Datos-Tesis/results/3a90ef36-98a4-11e9-a0d3-f4e9d4c27a82.h5", 300),
       picker_p_predict(file, 'standard_scaler', True, '/home/irene/Documents/Datos-Tesis/results/4fa7af9e-97a6-11e9-a0d3-f4e9d4c27a82.h5', 60),
       picker_p_predict(file, 'standard_scaler', False, '/home/irene/Documents/Datos-Tesis/results/5b77bc48-97eb-11e9-a0d3-f4e9d4c27a82.h5', 60)]
    for cnn in cnn_to_run:
        cnn

/home/irene/Documents/Datos-Tesis/Estaciones/2010.181.195432.SAC
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
picker_p: 2290
guess: 261
2019-06-29 19:16:55.244065
/home/irene/Documents/Resultados/01d880d6-9765-11e9-a0d3-f4e9d4c27a82/
/home/irene/Documents/Resultados/01d880d6-9765-11e9-a0d3-f4e9d4c27a82/OCM_06-29-2019::19-16.png
picker_p: 2290
guess: 0
2019-06-29 19:16:57.356973
/home/irene/Documents/Resultados/3a90ef36-98a4-11e9-a0d3-f4e9d4c27a82/
/home/irene/Documents/Resultados/3a90ef36-98a4-11e9-a0d3-f4e9d4c27a82/OCM_06-29-2019::19-16.png
picker_p: 2290
guess: 263
2019-06-29 19:16:58.895359
/home/irene/Documents/Resultados/4fa7af9e-97a6-11e9-a0d3-f4e9d4c27a82/
/home/irene/Documents/Resultados/4fa7af9e-97a6-11e9-a0d3-f4e9d4c27a82/OCM_06-29-2019::19-16.png
picker_p: 2290
guess: 264
2019-06-29 19:17:00.492777
/home/irene/Documents/Resultados/5b77bc48-97eb-11e9-a0d3-f4e9d4c27a82/
/home/irene/Documents/Resultados/

/home/irene/virtualenvs/obspy/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


picker_p: 12257
guess: 2354
2019-06-29 19:23:23.511151
/home/irene/Documents/Resultados/01d880d6-9765-11e9-a0d3-f4e9d4c27a82/
/home/irene/Documents/Resultados/01d880d6-9765-11e9-a0d3-f4e9d4c27a82/CALV_06-29-2019::19-23.png


In [ ]:
file_cnn = '/home/irene/Documents/Datos-Tesis/results/0b3c148e-977a-11e9-a0d3-f4e9d4c27a82.h5'
split = file_cnn.split("/")[6].split(".")
print(split[0])